In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from path import Path
import pickle
from prettytable import PrettyTable
from scipy.signal import find_peaks
plt.rcParams["font.family"] = "serif"
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)
graph_dpi = 100 

In [ ]:
# First run the test.py to create the results pickle file for analysis
architectures = ['cnn', 'fc', 'vit', 'rvit', 'rcnn']
datasets = ['mixed']
occlude_param = [None, "force_sensor", "robot_p", "robot_q", "robot_qd"]
data = ['rgb']
save_dir = Path('results')
dataset = {"dvrk": "dVRK", "dafoes": "DaFoEs", "mixed": "Mixed"}
param = {"force_sensor": "Force Sensor", "robot_p": "Robot Position", "robot_q": "Robot Joints", "robot_qd": "Robot Command"}

In [ ]:
# Load the data file or files
results = {}
for s in datasets:
    fig = plt.figure(figsize = (1720 / graph_dpi, 850 / graph_dpi), dpi = graph_dpi)
    fig.suptitle(f"Results for training over {dataset[s]}", fontsize=20, fontweight="bold")
    idx = 1
    for d in data:
        for occ in occlude_param:
            dafoes_mean_results, dafoes_std_results = [], []
            dvrk_mean_results, dvrk_std_results = [], []
            labels = []
            if occ != None: 
                ax = fig.add_subplot(1, len(occlude_param)-1, idx)
            for arch in architectures:
                if occ != None:
                    file_s = save_dir/"{}/{}/{}/{}_state_random.pkl".format(s, d, occ, arch)
                    labels.append(f"{arch.upper()}")
                else:
                    file_s = save_dir/"{}/{}/{}_state_random.pkl".format(s, d, arch)
                
                with open(file_s, 'rb') as fileobject:
                    results_s = pickle.load(fileobject)       
                try:
                    if occ != None:
                        dafoes_mean_results.append(results_s['dafoes_rmse_mean'].mean())
                        dafoes_std_results.append(results_s['dafoes_rmse_std'].mean())
                        dvrk_mean_results.append(results_s["dvrk_rmse_mean"].mean())
                        dvrk_std_results.append(results_s["dvrk_rmse_std"].mean())
                        results[f"{s}_{arch}_{occ}"] = results_s
                    else:
                        results[f"{s}_{arch}"] = results_s
                except KeyError:
                    continue
            
            if occ != None:
                x_pos_0 = np.arange(0, 2 * len(labels), 2)
                x_pos_1 = np.arange(1, 2 * len(labels) + 1, 2)

                ax.bar(x_pos_0, dafoes_mean_results, yerr=dafoes_std_results, color="purple", alpha=0.6, ecolor="black", error_kw=dict(lw=5, capsize=5, capthick=2))
                ax.bar(x_pos_1, dvrk_mean_results, yerr=dvrk_std_results, color="orange", alpha=0.6, ecolor='black', error_kw=dict(lw=5, capsize=5, capthick=2))
                ax.legend(["DaFoEs", "dVRK"], fontsize=16)

                if idx == 1:
                    ax.set_ylabel("RMSE (N)", fontsize = 16, fontweight="bold")

                
                ax.set_xticks(x_pos_0 + 0.5)
                ax.set_xticklabels(labels)

                ax.set_ylim([0.0, 0.8])
                ax.set_title(f"{param[occ]}", fontsize=16, fontweight="bold")

                idx += 1

        fig.tight_layout()
        fig.savefig(f"graphs/occlusion_{dataset[s]}", dpi=graph_dpi)
        plt.show()

In [ ]:
for occ in occlude_param:
    for arch in architectures:
        print(f"Values for occluded param: {occ} and {arch}")
        if occ != None:
            print(results[f"mixed_{arch}_{occ}"]["dvrk_rmse_std"].mean(), results[f"mixed_{arch}_{occ}"]["dvrk_rmse_mean"].mean())
        else:
            print(results[f"mixed_{arch}"]["dvrk_rmse_std"].mean(), results[f"mixed_{arch}"]["dvrk_rmse_mean"].mean())

In [ ]:
# Cell to plot the force estimation for the unseen testing set
my_dpi = 100
labels = ['X', 'Y', 'Z']

r = [int(30 * 10), int(30 * 20)]
# Zoomed areas of interest
# ranges = {"dvrk": [[int(30*10.5), int(30*11.5)], [30*12, int(30*13)], [int(30*85.5), int(30*86.5)]],
#           "dafoes": [[30*10, int(30*16)], [30*10, int(30*16)], [30*10, int(30*16)]],
#           "mixed": [[int(30*10.5), int(30*11.5)], [30*12, int(30*13)], [int(30*85.5), int(30*86.5)]]}



save_fig_root = Path('graphs')  
for s in datasets:
    x = (1/30)*np.linspace(0, len(results['{}_cnn_force_sensor'.format(s)]['dvrk_gt_mean']), len(results['{}_cnn_force_sensor'.format(s)]['dvrk_gt_mean']))
    x_rnn = (1/30)*np.linspace(0, len(results['{}_cnn_force_sensor'.format(s)]['dvrk_gt_mean']), len(results['{}_rcnn_force_sensor'.format(s)]['dvrk_pred_mean']))
    fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=False, sharey=False, figsize=(1920/my_dpi, 720/my_dpi), dpi=my_dpi)
    fig.supxlabel("Time (s)", fontsize=20, fontweight='bold')
    ax1.plot(x[r[0]:r[1]], results["{}_cnn".format(s)]['dvrk_gt_mean'][r[0]:r[1], 0], 'b', linewidth=6.0)
    ax1.plot(x[r[0]:r[1]], results["{}_cnn".format(s)]['dvrk_pred_mean'][r[0]:r[1], 0], 'g-.', linewidth=3.0)
    ax1.plot(x[r[0]:r[1]], results['{}_vit'.format(s)]['dvrk_pred_mean'][r[0]:r[1], 0], 'y-.', linewidth=3.0)
    ax1.plot(x[r[0]+4:r[1]+4], results['{}_rcnn'.format(s)]['dvrk_pred_mean'][r[0]:r[1], 0], 'r-.', linewidth=3.0)
    ax1.plot(x[r[0]+4:r[1]+4], results['{}_rvit'.format(s)]['dvrk_pred_mean'][r[0]:r[1], 0], 'k-.', linewidth=3.0)
    ax1.plot(x[r[0]:r[1]], results['{}_fc'.format(s)]['dvrk_pred_mean'][r[0]:r[1], 0], 'm-.', linewidth=3.0)
    ax1.set_ylabel("Force (N)", fontsize=18, fontweight='bold')
    # ax.legend(["GT", "CNN", "CNN-BAM", "RNN", "RNN-BAM", 'FC'], fontsize=14)
    ax1.legend(["GT"], fontsize=14, loc=2)

    ax2.plot(x[r[0]:r[1]], results["{}_cnn".format(s)]['dvrk_rmse_mean'][r[0]:r[1]], 'g-.', linewidth=3.0)
    ax2.plot(x[r[0]:r[1]], results['{}_vit'.format(s)]['dvrk_rmse_mean'][r[0]:r[1]], 'y-.', linewidth=3.0)
    ax2.plot(x[r[0]+4:r[1]+4], results['{}_rcnn'.format(s)]['dvrk_rmse_mean'][r[0]:r[1]], 'r-.', linewidth=3.0)
    ax2.plot(x[r[0]+4:r[1]+4], results['{}_rvit'.format(s)]['dvrk_rmse_mean'][r[0]:r[1]], 'k-.', linewidth=3.0)
    ax2.plot(x[r[0]:r[1]], results['{}_fc'.format(s)]['dvrk_rmse_mean'][r[0]:r[1]], 'm-.', linewidth=3.0)
    ax2.set_ylabel("RMSE (N)", fontsize=18, fontweight='bold')
    ax2.legend(["CNN", "VIT", "R-CNN", "R-VIT", "FC"], fontsize=14, loc=2)


    fig.align_labels()
    save_fig_path = save_fig_root
    save_fig_path.makedirs_p()
    fig.savefig(save_fig_path/f"{s}_error_evolution.png", dpi=my_dpi)
    fig.tight_layout()
    plt.show()

In [ ]:
my_dpi = 500
ranges = {"dvrk": [30*10, 30*13],
          "dafoes": [30*10, 30*13],
          "mixed": [30*84, 30*87]}

for s in datasets:
    x = (1/30)*np.linspace(0, len(results['{}_cnn_force_sensor'.format(s)]['test_gt']), len(results['{}_rgb_cnn_force_sensor_state'.format(s)]['test_gt']))
    x_rnn = (1/30)*np.linspace(0, len(results['{}_cnn_force_sensor_state'.format(s)]['test_gt']), len(results['{}_rgb_rcnn_force_sensor_state'.format(s)]['test_pred']))
    range_s = ranges[s]
    for op in occlude_param:
        if op is None:
            continue
        for d in data:
            r = range_s
            fig, ax = plt.subplots(ncols=1, nrows=1, sharex=False, sharey=False, figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
            fig.suptitle("{} evolution of RMSE for {} occlusion".format(s, op), fontsize=24, fontweight='bold')
            # ax.plot(x, results['{}_{}_cnn_state_{}'.format(s, d, f)]['test_gt'][r[0]:r[1], j], 'b', linewidth=3.0)
            ax.plot(x[r[0]:r[1]], results['{}_{}_cnn_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'g', linewidth=3.0)
            ax.plot(x[r[0]:r[1]], results['{}_{}_vit_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'y', linewidth=3.0)
            ax.plot(x[r[0]+4:r[1]+4], results['{}_{}_rcnn_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'r', linewidth=3.0)
            ax.plot(x[r[0]+4:r[1]+4], results['{}_{}_rvit_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'k', linewidth=3.0)
            ax.plot(x[r[0]:r[1]], results['{}_{}_fc_{}_state'.format(s, d, op)]['test_rmse'][r[0]:r[1]], 'm', linewidth=3.0)
            # ax.legend(["GT", "CNN", "CNN-BAM", "RNN", "RNN-BAM", 'FC'], fontsize=14)
            ax.legend(["CNN", "VIT", "R-CNN", "R-VIT", "FC"], fontsize=14, loc=1)
            ax.set_ylabel("RMSE (N)", fontsize=16, fontweight='bold')
            ax.set_xlabel("Time (s)", fontsize=16, fontweight='bold')
        
        save_fig_path = save_fig_root/"{}/{}".format(s, op)
        save_fig_path.makedirs_p()
        fig.align_labels()
        fig.savefig(save_fig_path/'metrics.png', dpi=my_dpi)
        plt.show()

In [ ]:
fig = plt.figure(figsize=(1920/my_dpi, 1080/my_dpi))
max_peaks_x, _ = find_peaks(results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 0], distance=150)
max_peaks_y, _ = find_peaks(results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 1], distance=150)
max_peaks_z, _ = find_peaks(results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 2], distance=150)

min_peaks_x, _ = find_peaks(-results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 0], distance=150)
min_peaks_y, _ = find_peaks(-results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 1], distance=150)
min_peaks_z, _ = find_peaks(-results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 2], distance=150)

plt.subplot(3, 1, 1)
plt.plot(x, results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 0], linewidth=2.0)
plt.plot(x[max_peaks_x], results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][max_peaks_x, 0], '*', markersize=10.0)
plt.plot(x[min_peaks_x], results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][min_peaks_x, 0], 'o', markersize=10.0)
plt.xlabel("Time (s)", fontsize=12)
plt.ylabel("Force X (N)", fontsize=12)
plt.legend(["Force", "Maxima", "Minima"], fontsize=12, loc=2)

plt.subplot(3, 1, 2)
plt.plot(x, results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 1], linewidth=2.0)
plt.plot(x[max_peaks_y], results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][max_peaks_y, 1], '*', markersize=10.0)
plt.plot(x[min_peaks_y], results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][min_peaks_y, 1], 'o', markersize=10.0)
plt.xlabel("Time (s)", fontsize=12)
plt.ylabel("Force Y (N)", fontsize=12)
plt.legend(["Force", "Maxima", "Minima"], fontsize=12, loc=2)

plt.subplot(3, 1, 3)
plt.plot(x, results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][:, 2], linewidth=2.0)
plt.plot(x[max_peaks_z], results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][max_peaks_z, 2], '*', markersize=10.0)
plt.plot(x[min_peaks_z], results["dvrk_rgb_cnn_force_sensor_state"]["test_gt"][min_peaks_z, 2], 'o', markersize=10.0)
plt.xlabel("Time (s)", fontsize=12)
plt.ylabel("Force Z (N)", fontsize=12)
plt.legend(["Force", "Maxima", "Minima"], fontsize=12, loc=2)

fig.align_labels()

fig.savefig("figures/max_min_force.png", dpi=my_dpi)

In [ ]:
fig = plt.figure()
for s in datasets:
    for d in data:
        for op in occlude_param:
            if op is None: continue
            peaks_x, _ = find_peaks(results["{}_{}_cnn_{}_state".format(s, d, op)]["dafoes_gt"][:, 0], distance=150)
            peaks_y, _ = find_peaks(results["{}_{}_cnn_{}_state".format(s, d, op)]["dafoes_gt"][:, 1], distance=150)
            peaks_z, _ = find_peaks(results["{}_{}_cnn_{}_state".format(s, d, op)]["dafoes_gt"][:, 2], distance=150)
            for arch in architectures:
                rmse_x = results["{}_{}_{}_{}_state".format(s, d, arch, op)]["dafoes_rmse"][peaks_x].mean()
                rmse_y = results["{}_{}_{}_{}_state".format(s, d, arch, op)]["dafoes_rmse"][peaks_y].mean()
                rmse_z = results["{}_{}_{}_{}_state".format(s, d, arch, op)]["dafoes_rmse"][peaks_z].mean()
                langs = ["X", "Y", "Z"]
                rmse = np.mean([rmse_x, rmse_y, rmse_z])
                print("Value for {}_{}_{}_{}: {} N".format(s, d, arch, op, rmse))
                # plt.bar(langs, rmse)
                # plt.pause(3.0)
                
